# Part 1: Inverted Index
- Create the Inverted Index
- Store it in a SQLite Database
- Compute $idf_t$ of each term in the Vocabulary
- Compute the corresponding $tfidf$ of each term/document combination
- Verify the computations were done correctly using the `scikit-learn` API library

In [1]:
import unit4.indexer as indexer
import unit4.indexer_storage as indexer_storage
from unit4.indexer_storage import SQLiteInvertedIndexStorage
from common.sqlite_utils import SQLiteDBManager
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

from IPython.core.display import HTML, Markdown

%load_ext autoreload
%autoreload 2

## Documents in the corpus

In [2]:
!ls ./unit4/cacm

CACM-0001.html CACM-0115.html CACM-0229.html CACM-0343.html CACM-0457.html
CACM-0002.html CACM-0116.html CACM-0230.html CACM-0344.html CACM-0458.html
CACM-0003.html CACM-0117.html CACM-0231.html CACM-0345.html CACM-0459.html
CACM-0004.html CACM-0118.html CACM-0232.html CACM-0346.html CACM-0460.html
CACM-0005.html CACM-0119.html CACM-0233.html CACM-0347.html CACM-0461.html
CACM-0006.html CACM-0120.html CACM-0234.html CACM-0348.html CACM-0462.html
CACM-0007.html CACM-0121.html CACM-0235.html CACM-0349.html CACM-0463.html
CACM-0008.html CACM-0122.html CACM-0236.html CACM-0350.html CACM-0464.html
CACM-0009.html CACM-0123.html CACM-0237.html CACM-0351.html CACM-0465.html
CACM-0010.html CACM-0124.html CACM-0238.html CACM-0352.html CACM-0466.html
CACM-0011.html CACM-0125.html CACM-0239.html CACM-0353.html CACM-0467.html
CACM-0012.html CACM-0126.html CACM-0240.html CACM-0354.html CACM-0468.html
CACM-0013.html CACM-0127.html CACM-0241.html CACM-0355.html CACM-0469.html
CACM-0014.html CACM-0128.

In [3]:
root_dir = "./unit4"
corpus_dir = root_dir+"/cacm"

In [4]:
indexer.LocalFileInvertedIndexer(storage=SQLiteInvertedIndexStorage(root_dir=root_dir)).reindex_directory(dirname=corpus_dir, verbose=False);

Start Time: 14:48
Indexing Complete, writing to storage (sqlite database): 14:48
Documents 570
Terms 2469
Tokens 9282
End Time: 14:48


In [5]:
inv_idx_sqldbm = SQLiteDBManager(root_dir+"/indexer_part2.db")

In [6]:
df_document_dict = inv_idx_sqldbm.sql_query_to_df("""
    SELECT 
        DocId, DocumentName 
    FROM 
        DocumentDictionary 
    ORDER BY 
        DocId
""").set_index('DocId')
df_document_dict

,DocumentName
DocId,
1,./unit4/cacm/CACM-0270.html
2,./unit4/cacm/CACM-0335.html
3,./unit4/cacm/CACM-0509.html
4,./unit4/cacm/CACM-0159.html
5,./unit4/cacm/CACM-0227.html
...,...
566,./unit4/cacm/CACM-0526.html
567,./unit4/cacm/CACM-0208.html
568,./unit4/cacm/CACM-0434.html


In [7]:
df_term_dict = inv_idx_sqldbm.sql_query_to_df("""
    SELECT 
        TermId, Term, N, df_t, idf_t 
    FROM 
        TermDictionary 
    ORDER BY 
        TermId
""").set_index('TermId')
df_term_dict

,Term,N,df_t,idf_t
TermId,,,,
1,techniqu,570,30,2.944439
2,storag,570,21,3.301114
3,alloc,570,14,3.706579
4,algorithm,570,217,0.965739
5,cacm,570,570,0.000000
...,...,...,...,...
2465,ca600403,570,1,6.345636
2466,poor,570,1,6.345636
2467,ca621205,570,1,6.345636


In [8]:
df_posting = inv_idx_sqldbm.sql_query_to_df("""
    SELECT 
        TermId, DocId, tf_t_d, idf_t, tfidf 
    FROM 
        Posting 
    ORDER BY 
        TermId, DocId
""").set_index(['TermId', 'DocId'])
df_posting

tf_t_d     idf_t     tfidf
TermId DocId                            
1      1           1  2.944439  2.944439
       6           1  2.944439  2.944439
       19          3  2.944439  8.833317
       26          1  2.944439  2.944439
       39          1  2.944439  2.944439
...              ...       ...       ...
2465   567         1  6.345636  6.345636
2466   568         1  6.345636  6.345636
2467   568         1  6.345636  6.345636
2468   569         1  6.345636  6.345636
2469   570         1  6.345636  6.345636

[8154 rows x 3 columns]

### Now the question is: ***was $idf_t$ and consequently $tfidf$ computed correctly?***
To answer this question, I will use the tried and tested `scikit-learn` (usually abbreviated `sklearn`) library ("scikit-learn: machine learning in Python — scikit-learn 0.23.1 documentation", 2020), used in Data Science and Machine Learning for years now, to compute $idf_t$ and then compare it to my results.

<p><br>
    
## Verify correctness of $idf_t$ computation using `sklearn`'s `TfidfTransformer` for comparison
    
`sklearn`'s `TfidfTransformer` ("sklearn.feature_extraction.text.TfidfTransformer — scikit-learn 0.23.1 documentation", 2020) takes a word-count vector as input, provided by fitting/transforming the corpus of documents with `sklearn`'s `CountVectorizer` ("sklearn.feature_extraction.text.CountVectorizer — scikit-learn 0.23.1 documentation", 2020).
    
`sklearn`'s `CountVectorizer` takes the vector of documents - i.e. the corpus in vector form - as input.
    
So, as a preliminary step, the corpus itself needs to be vectorized - i.e. each document literally needs to be loaded as a component of this vector.
    
Note that this means we load the entire corpus into RAM.  
    
It is important to understand that in a production environment, this is exactly what we want to avoid doing since a corpus of documents is usually far too large to fit into RAM!  
But, the whole point of this exercise is to verify that the computation of $idf_t$ was done correctly.  In other words, this is not something we would do in a production environment.  Once we have proven that this computation was, in fact, done correctly, this will never need to be again.  Think of it as the loose equivalent of a mathematical proof.
    
I'll re-use some functionality I used in creating the inverted index above.  Namely, I want to do the same preprocessing done in that case.  That functionality is housed within the `InvertedIndexDS` class, in the indexer_ds.py Python source code file.
    
The functionality below does not actually store the inverted index but it does do the same preprocessing as the above.  It also uses the `__walk_directory__` functionality adapted from our "reference" source-code.  The bottom line is that I extract the "corpus vector" of preprocessed documents so that `sklearn`'s `CountVectorizer` and `TfidfTransformer` can be used to compare its $idf_t$ computations to mine.

In [9]:
class FakeInvertedIndexStorage(indexer_storage.AbstractInvertedIndexStorage):
    def __init__(self):
        super().__init__("", "INDEX IS NOT ACTUALLY STORED")
        
    def init_storage(self):
        # do nothing since nothing will actually be stored
        print(f"Message from {self.__class__.__name__}: Hello there. I promise I will not store anything (and certainly will not overwrite any existing inverted index storage)!")
        return
    
    def insert_doc_dict(self, doc_path, doc_id):
        # do nothing since nothing will actually be stored
        return
    
    def insert_term_dict(self, term, termid, N, df_t, idf_t):
        # do nothing since nothing will actually be stored
        return
    
    def insert_posting(self, termid, docid, tf_t_d, idf_t, tfidf):
        # do nothing since nothing will actually be stored
        return
    
    def finalize_storage(self):
        # do nothing since nothing will actually be stored
        return
    
class CustomLocalFileInvertedIndexer(indexer.LocalFileInvertedIndexer):
    def __init__(self, skip_preprocessing=False):
        self.skip_preprocessing = skip_preprocessing
        self.corpus_vector = []
        super().__init__(storage=FakeInvertedIndexStorage())
        
    # override this from base class; in LocalFileInvertedIndexer this function does nothing with the vector of tokens
    #    but of course it does update the inverted index data structure (InvertedIndexDS), which we still want to do
    #    but we want to use the vector of tokens after a document is preprocessed and tokenized
    def process_local_file(self, file):
        # the original version in the baseclass does this:
#         for l in file.readlines(): 
#             self.ds.parse_tokenize(l)

        if not self.skip_preprocessing:
            # but we want ours to do this (when skip_preprocessing is False):
            preprocessed_lines = []
            for l in file.readlines(): 
                preprocessed_lines.append(' '.join(self.ds.parse_tokenize(l)))
            self.corpus_vector.append(' '.join(preprocessed_lines)) # add the preprocessed version of the document (as one long string) to the corpus vector
        else:
            # otherwise, let's leave the document as is (so we can inspect by comparison what our documents actually look like AFTER preprocessing)
            self.corpus_vector.append(file.read())
    
    # also override this so that we can reset corpus_vector when skip_preprocessing is changed without having to reinstantiate a new object
    def reindex_directory(self, dirname, verbose=False):
        self.corpus_vector = []
        return super().reindex_directory(dirname, verbose)

In [10]:
custom_localfile_indexer = CustomLocalFileInvertedIndexer(skip_preprocessing=True)
custom_localfile_indexer.reindex_directory(dirname=corpus_dir) # remember that this has "fake" storage - i.e. nothing is actually getting stored (so the real inverted index stored in the sqlite db is intact

docs_not_preprocessed = custom_localfile_indexer.corpus_vector
display(Markdown(f"count docs: {len(docs_not_preprocessed)}"))
display(Markdown(f"### For example, the first document BEFORE it is (pre)processed:"))
print(docs_not_preprocessed[0])

Message from FakeInvertedIndexStorage: Hello there. I promise I will not store anything (and certainly will not overwrite any existing inverted index storage)!
Start Time: 14:48
Indexing Complete, writing to storage (INDEX IS NOT ACTUALLY STORED): 14:48
*** WARNING!!! Apparently none of the <unit4.indexer_ds.InvertedIndexDS object at 0x7fde0a7c31f0> terms occurs in more than one document out of the entire collection of 570 documents!!! ***
Documents 570
Terms 0
Tokens 0
End Time: 14:48


count docs: 570

### For example, the first document BEFORE it is (pre)processed:

<html>
<pre>


Techniques for Storage Allocation Algorithms 

CACM October, 1961

Kelley Jr., J. E.

CA611011 JB March 16, 1978  12:50 PM

270	5	270
270	5	270
270	5	270
678	5	270
270	6	270

</pre>
</html>



<p><br><br>

Note the scary warning that is output above.  This is entirely expected since no preprocessing (and consequently no tokenization) is done when `CustomLocalFileInvertedIndexer.skip_preprocessing==True`.  The effect is that, of course, as expected, the `InvertedIndexDS` is not updated (aside from updating its `InvertedIndexDS.n_documents` member, it is never touched).  But that is beside the point.

The point in showing the above was simply "bonus material" to be able to see the "before" vs "after" effects of preprocessing/tokenization.

In [11]:
custom_localfile_indexer.skip_preprocessing = False
custom_localfile_indexer.reindex_directory(dirname=corpus_dir) # remember that this has "fake" storage - i.e. nothing is actually getting stored (so the real inverted index stored in the sqlite db is intact

docs_preprocessed = custom_localfile_indexer.corpus_vector
display(Markdown(f"count docs: {len(docs_preprocessed)}"))
display(Markdown(f"### For example, the first document AFTER it is (pre)processed:"))
print(docs_preprocessed[0])

Message from FakeInvertedIndexStorage: Hello there. I promise I will not store anything (and certainly will not overwrite any existing inverted index storage)!
Start Time: 14:48
Indexing Complete, writing to storage (INDEX IS NOT ACTUALLY STORED): 14:48
Documents 570
Terms 2469
Tokens 9282
End Time: 14:48


count docs: 570

### For example, the first document AFTER it is (pre)processed:

    techniqu storag alloc algorithm  cacm octob  kellei  ca611011 march         


<p><br><br>
    
Now we can actually move on to the guts of this effort: to validate the computation of $idf_t$ was done correctly by comparing the result from the appropriate `sklearn` classes/APIs...

In [12]:
cv = CountVectorizer(vocabulary=df_term_dict.Term.values)
preprocessed_word_count_vector = cv.fit_transform(docs_preprocessed)

In [13]:
tfidf_transformer = TfidfTransformer(use_idf=True, smooth_idf=False, norm=None, sublinear_tf=False)
tfidf_mat = tfidf_transformer.fit_transform(preprocessed_word_count_vector)

There are two things to note from the above (using `TfidfTransformer`), based on the official `sklearn` documentation as well as our textbook:
1. *`sklearn.TfidfTransformer` documentation*: "... the idf is computed as idf(t) = log [ n / df(t) ] + 1 (if smooth_idf=False)"
2. *paraphrased from our textbook*: the formula is $idf_t = log_{10}(\frac{N}{df_t})$ (note the base here is 10) but the authors go on to say that **the base is irrelevant** (Manning, C., Raghavan, P., & Schütze, H., 2008, p. 118)

So... since the goal here is to utilize `TfidTransformer` to confirm that my computation of $idf_t$ was done correctly, one of the two computations needs to be adjusted to be scaled and translated to the other.

Thus:
1. in my computation of $idf_t$, I use the same base used in `sklearn`'s `TfidfTransformer` $idf_t$ computation, which is base $e$ - i.e. I use $idf_t = log_{e}(\frac{N}{df_t})$ to match `sklearn`'s `TfidfTransformer`
2. since `sklearn`'s `TfidfTransformer` $idf_t$ computation translates $log_{e}(\frac{N}{df_t})$ by 1 (adds 1), when comparing my computed $idf_t$, I subtract 1 from`sklearn`'s `TfidfTransformer` $idf_t$ computation

Finally, **if I have coded this assignment correctly, after adjusting according to the above, the two computations of $idf_t$ be identical for EVERY term**.

In [14]:
df_idf__inv_idx = pd.DataFrame(
    {
        'term_id__inv_idx': df_term_dict.reset_index().TermId,
        'idf_t__inv_idx': df_term_dict.reset_index().idf_t
    }
).set_index('term_id__inv_idx')

df_idf__skl = pd.DataFrame(
    {
        'term_id__skl': list(range(1, len(tfidf_transformer.idf_)+1)), 
        'idf_t__skl': tfidf_transformer.idf_-1  # from skl docs: "the idf is computed as idf(t) = log [ n / df(t) ] + 1 (if smooth_idf=False)", SO WE MUST SUBTRACT 1
    }
).set_index('term_id__skl')

df_idf_comparison = pd.concat([df_term_dict[['Term']], df_idf__inv_idx, df_idf__skl], axis=1, join='inner')

# add a new column that holds the difference between the two
df_idf_comparison['delta'] = np.abs(df_idf_comparison.idf_t__inv_idx - df_idf_comparison.idf_t__skl)

In [15]:
# finally, list rows from the above dataframe where idf_t__inv_idx != idf_t__skl
df_difference = df_idf_comparison.query(f"delta > 0")
df_difference

,Term,idf_t__inv_idx,idf_t__skl,delta
21,equat,3.167583,3.167583,4.440892e-16
81,ibm,3.455265,3.455265,4.440892e-16
98,digit,3.049799,3.049799,4.440892e-16
135,on,3.167583,3.167583,4.440892e-16
191,data,3.049799,3.049799,4.440892e-16
217,discuss,3.455265,3.455265,4.440892e-16
266,polynomi,3.167583,3.167583,4.440892e-16
305,compil,3.210142,3.210142,4.440892e-16
432,algol,3.049799,3.049799,4.440892e-16
433,januari,3.167583,3.167583,4.440892e-16


In [16]:
len(df_difference)

15

So there are evidently 15 terms out of 2469 from the vocabulary where the computation differs.

But look at the difference: that value is a decimal-point followed by 16 zeroes.

Explanation: floating-point rounding error has occurred.

CONCLUSION: **$idf_t$ was computed correctly**.

<p><br><br>

# Conclusion
I forgot to mention that this document was produced as a (IPython) Jupyter Notebook. ("Project Jupyter", 2020)

### References

scikit-learn: machine learning in Python — scikit-learn 0.23.1 documentation. (2020). Retrieved from https://scikit-learn.org/stable/index.html

sklearn.feature_extraction.text.CountVectorizer — scikit-learn 0.23.1 documentation. (2020). Retrieved from https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

sklearn.feature_extraction.text.TfidfTransformer — scikit-learn 0.23.1 documentation. (2020). Retrieved from https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html

Manning, C., Raghavan, P., & Schütze, H. (2008). Introduction to Information Retrieval [Ebook]. Cambridge University Press. Retrieved from http://nlp.stanford.edu/IR-book/pdf/irbookprint.pdf

Project Jupyter. (2020). Retrieved from https://jupyter.org/documentation